curl -fsSL https://ollama.com/install.sh | sh

ollama serve => 실행해서 진행하기

pip install requests

In [1]:
# 모델 목록 조회
import requests

url = "http://localhost:11434/api/models/"
response = requests.get(url)

if response.status_code == 200:
    print("Available models:", response.json())
else:
    print(f"Error {response.status_code}: {response.text}")

Error 404: 404 page not found


In [23]:
import os
import logging
import traceback
import tempfile
from functools import lru_cache

import torch
from pydub import AudioSegment
from pydub.effects import normalize
import whisper

# Whisper 모델을 전역으로 한 번만 로드
whisper_model = whisper.load_model("medium")

def filter_segments(segments, noise_keywords):
    """
    노이즈 키워드를 제거하는 함수
    """
    return [
        segment["text"] for segment in segments 
        if not any(keyword.lower() in segment["text"].lower() for keyword in noise_keywords)
    ]

@lru_cache(maxsize=10)
def transcribe_audio(file_path):
    """
    오디오 파일을 텍스트로 변환하는 캐싱 함수
    """
    return whisper_model.transcribe(
        file_path, 
        word_timestamps=False,
        fp16=torch.cuda.is_available(),
        language = 'ko'
    )

def voice_to_text(maf_idx, file_path):
    """
    AWS 서버에서 파일 경로와 maf_idx를 가져와 음성 파일을 텍스트로 변환하고 결과를 반환하는 함수
    """
    # 로깅 설정
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s: %(message)s")
    logger = logging.getLogger(__name__)

    # 결과 저장용 변수
    result = {'maf_idx': maf_idx, 'voice_to_text': None}

    try:
        # 임시 디렉토리 사용
        with tempfile.TemporaryDirectory() as tmpdir:
            print(f"Processing file: {file_path}")

            # 1. 파일 변환 (webm -> wav)
            file_name = os.path.splitext(os.path.basename(file_path))[0]
            converted_file = os.path.join(tmpdir, f"{file_name}_converted.wav")
            audio = AudioSegment.from_file(file_path)
            print(f"\t\t 파일 길이: {len(audio) / 1000} 초")  # 길이를 초 단위로 출력
            audio.export(converted_file, format="wav")
            logger.info(f"WAV 변환 완료: {converted_file}")

            # 2. 오디오 정규화
            normalized_file = os.path.join(tmpdir, f"{file_name}_normalized.wav")
            normalized_audio = normalize(AudioSegment.from_file(converted_file))
            normalized_audio.export(normalized_file, format="wav")
            logger.info(f"오디오 정규화 완료: {normalized_file}")

            # 3. Whisper로 텍스트 변환
            logger.info(f"Whisper 모델로 텍스트 변환 시작: {normalized_file}")
            transcription_result = transcribe_audio(normalized_file)
            logger.info("텍스트 변환 완료")

            # 4. 텍스트 병합 및 노이즈 키워드 제거
            noise_keywords = ["전화기 소리", "기침 소리", "잡음", "배경 소음"]
            segments = filter_segments(transcription_result.get("segments", []), noise_keywords)
            combined_text = " ".join(segments)

            # 결과 저장
            result['voice_to_text'] = combined_text

    except Exception as e:
        logger.error(f"Error processing file {file_path}")
        logger.error(traceback.format_exc())
        result['voice_to_text'] = f"Error processing file: {e}"

    return result
if __name__ == "__main__":
    file = "/home/yjtech/Desktop/LLM/sample_audio.webm"
    text_res = voice_to_text(1, file)

/home/yjtech/Desktop/conda/anaconda3/envs/venv38-llm/lib/python3.8/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.lo

Processing file: /home/yjtech/Desktop/LLM/sample_audio.webm


2024-12-10 06:30:31,671 - INFO: WAV 변환 완료: /tmp/tmp166zrnu_/sample_audio_converted.wav
2024-12-10 06:30:31,693 - INFO: 오디오 정규화 완료: /tmp/tmp166zrnu_/sample_audio_normalized.wav
2024-12-10 06:30:31,693 - INFO: Whisper 모델로 텍스트 변환 시작: /tmp/tmp166zrnu_/sample_audio_normalized.wav


		 파일 길이: 46.695 초


2024-12-10 06:30:52,916 - INFO: 텍스트 변환 완료


In [24]:
text_res

{'maf_idx': 1,
 'voice_to_text': ' 안녕하세요. 포항시 오천읍에 거주하는 이강일입니다.  저는 지난 11월 13일 이후로 집에서 나오는 수돗물에서 고약한 냄새  가 나기 시작했습니다. 처음에는 아파트 물탱크의 문제가  있는 줄 알고 대수롭지 않게 여 겠는데 냄새가 계속 나서 걱정이  돼서 신고합니다. 현재 수돗물에서 흙냄새와 곰팡이  냄새가 나고 그래서 설거지나 세수 는 물론 물을 마실 수가 없습니다.  생수로 대체해서 사용하고 있는데 이 문제로 매우 불편을 겪고 있습니다.  포항시 수돗물 원수의 40%를 공급 하는 경주 안개 때문에서 녹조  현상이 발생했다고 들었고 남조류 에서 발생한 지오스민이 냄새를  유발한다고 합니다. 이 문제에 대한 빠른 대응을 부탁  드립니다. 감사합니다.'}

In [4]:
import requests
import json
import re

def ensure_list(value):
    """
    입력값이 리스트가 아니면 리스트로 변환하고, 입력값이 None이면 빈 리스트를 반환합니다.
    """
    if value is None:
        return []
    return [value] if not isinstance(value, list) else value

def extract_information(maf_idx, spelling_text_res):
    """
    Ollama API를 호출하여 문장에서 시간, 장소, 냄새 종류, 냄새 장소, 냄새 강도 등을 추출합니다.
    """
    result = {
        'maf_idx': maf_idx, 
        'voice_to_text': spelling_text_res['voice_to_text'], 
        'keywords': None
    }
    
    # Ollama API URL
    url = "http://localhost:11434/api/generate"
    
    # Request payload
    payload = {
        "model": "llama3",
        "prompt": f"""
            아래 문장에서 '시간', '장소', '냄새 종류', '냄새 장소', '냄새 강도(표현)'을 각각 추출하세요. 
            다음과 같은 형식으로 JSON으로만 반환하세요. 설명은 포함하지 마세요.
            - '시간': 문장에서 명확히 시간임을 나타내는 단어/구 표현(예: 퇴근 후, 새벽, 아침 등)과, 가능한 경우 절대적인 시간 정보(예: 새벽 4시, 오후 6시 등)를 함께 추출하세요.(여러 개일 경우 모두 포함)
                - 상대적인 표현(예: 퇴근 후, 점심시간 등)은 그대로 유지하세요.
                - 절대적인 시간 표현이 문장에 포함된 경우 이를 구체적으로 포함하세요.
            - '장소': 문장에서 언급된 사람이 거주하는 지역이나 큰 지리적 위치(예: 도시명, 마을명 등)(여러 개일 경우 모두 포함)
            - '냄새 종류': 문장에서 언급된 냄새의 구체적인 종류(여러 개일 경우 모두 포함)
            - '냄새 장소': 냄새가 발생한 구체적인 위치(예: 아파트 동/호수, 거리명, 수돗물, 화장실 등)(여러 개일 경우 모두 포함)
            - '냄새 강도': 냄새 강도를 나타내는 표현(여러 개일 경우 모두 포함)

        문장: "{spelling_text_res['voice_to_text']}"

        형식:
        {{

            "time": "추출된 시간",
            "location": "사람이 거주하는 지역",
            "smell_type": "추출된 냄새 종류",
            "smell_location": "냄새가 발생한 구체적인 위치",
            "smell_intensity": "추출된 냄새 강도"
        }}
        """
    }

    try:
        # API call
        response = requests.post(url, json=payload, stream=True)
        
        if response.status_code != 200:
            result['keywords'] = {
                'error': f"API Error {response.status_code}: {response.text}"
            }
            return result

        full_response = ""
        for line in response.iter_lines(decode_unicode=True):
            if line:
                data = json.loads(line)
                full_response += data.get("response", "")
        
        # JSON 포맷을 추출하는 정규 표현식
        json_match = re.search(r'```json\n(.*?)\n```', full_response, re.DOTALL)
        
        if json_match:
            try:
                # JSON 문자열을 파싱하여 키워드 추출
                keywords = json.loads(json_match.group(1))
                
                result['keywords'] = {
                    'time': ensure_list(keywords.get('time', '')),
                    'location': ensure_list(keywords.get('location', '')),
                    'smell_type': ensure_list(keywords.get('smell_type', '')),
                    'smell_location': ensure_list(keywords.get('smell_location', '')),
                    'smell_intensity': ensure_list(keywords.get('smell_intensity', ''))
                }
            except json.JSONDecodeError:
                result['keywords'] = {
                    'error': 'Failed to parse JSON',
                    'raw_response': full_response
                }
        else:
            # 만약 JSON이 백틱(```)으로 감싸져 있지 않다면 일반적인 JSON 객체로 시도
            json_match = re.search(r'\{[^}]+\}', full_response)
            if json_match:
                try:
                    keywords = json.loads(json_match.group(0))
                    result['keywords'] = {
                        'time': ensure_list(keywords.get('time', '')),
                        'location': ensure_list(keywords.get('location', '')),
                        'smell_type': ensure_list(keywords.get('smell_type', '')),
                        'smell_location': ensure_list(keywords.get('smell_location', '')),
                        'smell_intensity': ensure_list(keywords.get('smell_intensity', ''))
                    }
                except json.JSONDecodeError:
                    result['keywords'] = {
                        'error': 'Failed to parse JSON',
                        'raw_response': full_response
                    }
            else:
                result['keywords'] = {
                    'error': 'No JSON found',
                    'raw_response': full_response
                }
        
        return result

    except Exception as e:
        result['keywords'] = {
            'error': str(e)
        }
        return result


In [5]:
maf_idx = text_res['maf_idx']

key_word_res = extract_information(maf_idx, text_res)
key_word_res

{'maf_idx': 1,
 'voice_to_text': ' 안녕하세요. 포항시 오천읍에 거주하는 이강일입니다.  저는 지난 11월 13일 이후로 집에서 나오는 수돗물에서 고약한 냄새  가 나기 시작했습니다. 처음에는 아파트 물탱크의 문제가  있는 줄 알고 대수롭지 않게 여 겠는데 냄새가 계속 나서 걱정이  돼서 신고합니다. 현재 수돗물에서 흙냄새와 곰팡이  냄새가 나고 그래서 설거지나 세수 는 물론 물을 마실 수가 없습니다.  생수로 대체해서 사용하고 있는데 이 문제로 매우 불편을 겪고 있습니다.  포항시 수돗물 원수의 40%를 공급 하는 경주 안개 때문에서 녹조  현상이 발생했다고 들었고 남조류 에서 발생한 지오스민이 냄새를  유발한다고 합니다. 이 문제에 대한 빠른 대응을 부탁  드립니다. 감사합니다.',
 'keywords': {'time': ['11월 13일', '지금'],
  'location': ['포항시 오천읍', '포항시'],
  'smell_type': ['고약한 냄새', '흙냄새', '곰팡이 냄새'],
  'smell_location': ['수돗물', '아파트 물탱크'],
  'smell_intensity': ['가 나기 시작', '나가서']}}

In [1]:
import requests
import json
import re
from functools import lru_cache

def ensure_list(value):
    """
    입력값이 리스트가 아니면 리스트로 변환하고, 입력값이 None이면 빈 리스트를 반환합니다.
    """
    if value is None:
        return []
    return [value] if not isinstance(value, list) else value

@lru_cache(maxsize=32)
def create_ollama_prompt(text):
    """
    프롬프트 캐싱을 통해 반복 호출 시 메모리와 처리 시간 최적화
    """
    return f"""
        아래 문장에서 '시간', '장소', '냄새 종류', '냄새 장소', '냄새 강도(표현)'을 각각 추출하세요. 
        다음과 같은 형식으로 JSON으로만 반환하세요. 설명은 포함하지 마세요.
        - '시간': 문장에서 명확히 시간임을 나타내는 단어/구 표현(예: 퇴근 후, 새벽, 아침 등)과, 가능한 경우 절대적인 시간 정보(예: 새벽 4시, 오후 6시 등)를 함께 추출하세요.
            - 상대적인 표현(예: 퇴근 후, 점심시간 등)은 그대로 유지하세요.
            - 절대적인 시간 표현이 문장에 포함된 경우 이를 구체적으로 포함하세요.
            - 여러 시간이 언급되면 중복없이 모두 추출하세요.
        - '장소': 문장에서 언급된 사람이 거주하는 지역이나 큰 지리적 위치(예: 도시명, 마을명 등)
            - 여러 장소가 언급되면 중복없이 모두 추출하세요.
        - '냄새 종류': 문장에서 언급된 냄새의 구체적인 종류
            - 명확한 냄새의 이름과 성격을 포함하세요.
            - 여러 냄새 종류가 언급되면 중복없이 모두 추출하세요.
        - '냄새 장소': 냄새가 발생한 구체적인 위치(예: 아파트 동/호수, 거리명, 수돗물, 화장실 등)
            - 여러 냄새장소가 언급되면 중복없이 모두 추출하세요.
        - '냄새 강도': 냄새 강도를 나타내는 표현
            - 냄새와 관련된 감정이나 상태를 표현하는 단어를 포함하세요.
            - 여러 냄새 강도가 언급되면 중복없이 모두 추출하세요.

    문장: "{text}"

    형식:
    {{
        "time": "추출된 시간",
        "location": "사람이 거주하는 지역",
        "smell_type": "추출된 냄새 종류",
        "smell_location": "냄새가 발생한 구체적인 위치",
        "smell_intensity": "추출된 냄새 강도"
    }}
    """

def parse_ollama_response(full_response):
    """
    Ollama 응답 파싱 로직 분리로 코드 가독성 및 메모리 효율성 개선
    """
    # JSON 포맷을 추출하는 정규 표현식
    json_match = (
        re.search(r'```json\n(.*?)\n```', full_response, re.DOTALL) or 
        re.search(r'\{[^}]+\}', full_response)
    )
    
    if not json_match:
        return {'error': 'No JSON found', 'raw_response': full_response}
    
    try:
        keywords = json.loads(json_match.group(1) if '```json' in full_response else json_match.group(0))
        return {
            'time': ensure_list(keywords.get('time', '')),
            'location': ensure_list(keywords.get('location', '')),
            'smell_type': ensure_list(keywords.get('smell_type', '')),
            'smell_location': ensure_list(keywords.get('smell_location', '')),
            'smell_intensity': ensure_list(keywords.get('smell_intensity', ''))
        }
    except json.JSONDecodeError:
        return {'error': 'Failed to parse JSON', 'raw_response': full_response}

def extract_information(maf_idx, spelling_text_res):
    """
    Ollama API를 호출하여 문장에서 시간, 장소, 냄새 종류, 냄새 장소, 냄새 강도 등을 추출합니다.
    """
    result = {
        'maf_idx': maf_idx, 
        'voice_to_text': spelling_text_res['voice_to_text'], 
        'keywords': None
    }
    
    # Ollama API URL
    url = "http://61.37.153.212:11434/api/generate"
    
    # Request payload
    payload = {
        "model": "llama3",
        "prompt": create_ollama_prompt(spelling_text_res['voice_to_text'])
    }

    try:
        # API call
        with requests.post(url, json=payload, stream=True) as response:
            if response.status_code != 200:
                result['keywords'] = {
                    'error': f"API Error {response.status_code}: {response.text}"
                }
                return result

            full_response = ''.join(
                data.get("response", "") 
                for line in response.iter_lines(decode_unicode=True) 
                if line and (data := json.loads(line))
            )
        
        # 응답 파싱
        result['keywords'] = parse_ollama_response(full_response)
        
        return result

    except Exception as e:
        result['keywords'] = {
            'error': str(e)
        }
        return result

In [1]:
import requests
import json
import re

def ensure_list(value):
    """
    입력값이 리스트가 아니면 리스트로 변환하고, 입력값이 None이면 빈 리스트를 반환합니다.
    """
    if value is None:
        return []
    return [value] if not isinstance(value, list) else value

def validate_keywords(original_text, extracted_keywords):
    """
    원문에 포함된 키워드인지 검증합니다.
    모든 키워드(time, location, smell_type, smell_intensity)에 대해 원문에서 명시적으로 존재하는 경우만 반환합니다.
    """
    validated_keywords = {}
    for key, values in extracted_keywords.items():
        if isinstance(values, list):
            # 키워드가 리스트일 경우: 원문에 존재하는 단어만 필터링
            validated_keywords[key] = [val for val in values if val in original_text]
        elif isinstance(values, str):
            # 키워드가 문자열일 경우: 원문에 포함된 경우에만 유지
            validated_keywords[key] = values if values in original_text else None
    return validated_keywords


def extract_information(maf_idx, spacing_spelling_res):
    """
    Ollama API를 호출하여 문장에서 시간, 장소, 냄새 종류, 냄새 장소, 냄새 강도 등을 추출합니다.
    """
    result = {
        'maf_idx': maf_idx, 
        'voice_to_text': spacing_spelling_res['voice_to_text'], 
        'keywords': None
    }
    
    # Ollama API URL
    url = "http://61.37.153.212:11434/api/generate"
    
    # Request payload
    payload = {
        "model": "llama3:70b",
        "temperature": 0.0,  # 완전히 결정적으로 설정 -> 낮을수록 창의성이 낮음
        "top_p": 1.0,       # 확률 분포 전체 사용 -> 낮을수록 모델의 예측이 보수적
        "repeat_penalty": 1, # 택스트에서 단어나 구가 반복되는 것을 억제하는 역할 -> 높을수록 같은 단어가 계속 반복되는 문제를 방지
        "seed": 2024,  # seed 값을 추가하여 결정적 결과를 강제
        "cache_disabled": True,  # 캐싱 비활성화 옵션
        "prompt": f"""
            너는 민원 데이터를 사용해 중요한 키워드를 추출하는거야
            문장에서 시간, 장소, 냄새 종류, 냄새 강도를 JSON 형식으로 추출해주세요.
            [문장]: "{spacing_spelling_res['spacing_and_spelling_res']}"

            형식:
            {{
                "time": ["시간표현1", "시간표현2"],
                "location": ["장소"],
                "smell_type": ["냄새종류1", "냄새종류2"],
                "smell_intensity": ["강도표현1", "강도표현2"]
            }}

            규칙:
            1. [시간 추출 규칙]:
            - 문장에서 직접적으로 언급된 시간 표현만 포함.
            - 절대적 시간: "오전 9시", "저녁 7시"
            - 상대적 시간: "퇴근", "아침", "저녁"
            - 문장 내에 복합적으로 표현된 시간도 모두 포함.
            - 추론하지 말 것.
            2. [장소 추출 규칙]:
            - 구체적인 위치나 거주지 정보만 포함.
            - 아파트명, 동, 호수 등 상세 정보 포함.
            - 문장 내 민원인의  위치를 추출
            - 예를 들어 힐스테이트 포항, 아이파크, 오천힐스테이트 등등등
            3. [냄새 종류 추출 규칙]:
            - 냄새 종류와 발생 위치를 포함한 구체적 표현만 추출.
            - 문장 내 냄새 종류와 발생 위치를 추출
            4. [냄새 강도 추출 규칙]:
            - 강도를 나타내는 형용사 또는 부사 표현 포함.
            - 문장 내 냄새 강도를 나타내는 표현을추출
            """
    }


    try:
        # API call
        response = requests.post(url, json=payload, timeout=60)
        response.raise_for_status()  # HTTP 오류 상태 코드 확인
        
        # API 응답 처리
        full_response = ""
        for line in response.iter_lines(decode_unicode=True):
            if line:
                data = json.loads(line)
                full_response += data.get("response", "")
        
        # JSON 포맷을 추출하는 정규 표현식
        json_match = re.search(r'```json\n(.*?)\n```', full_response, re.DOTALL)
        if json_match:
            try:
                # JSON 문자열을 파싱하여 키워드 추출
                keywords = json.loads(json_match.group(1))
                validated_keywords = validate_keywords(spacing_spelling_res['voice_to_text'], {
                    'time': ensure_list(keywords.get('time', [])),
                    'location': ensure_list(keywords.get('location', ''))[0],
                    'smell_type': ensure_list(keywords.get('smell_type', [])),
                    'smell_intensity': ensure_list(keywords.get('smell_intensity', []))
                })
                result['keywords'] = validated_keywords
            except json.JSONDecodeError:
                result['keywords'] = {
                    'error': 'Failed to parse JSON',
                    'raw_response': full_response
                }
        else:
            # 만약 JSON이 백틱(```)으로 감싸져 있지 않다면 일반적인 JSON 객체로 시도
            json_match = re.search(r'\{[^}]+\}', full_response)
            if json_match:
                try:
                    keywords = json.loads(json_match.group(0))
                    
                    validated_keywords = validate_keywords(spacing_spelling_res['voice_to_text'], {
                        'time': ensure_list(keywords.get('time', [])),
                        'location': ensure_list(keywords.get('location', ''))[0],
                        'smell_type': ensure_list(keywords.get('smell_type', [])),
                        'smell_intensity': ensure_list(keywords.get('smell_intensity', []))
                    })
                    result['keywords'] = validated_keywords
                except json.JSONDecodeError:
                    result['keywords'] = {
                        'error': 'Failed to parse JSON',
                        'raw_response': full_response
                    }
            else:
                result['keywords'] = {
                    'error': 'No JSON found',
                    'raw_response': full_response
                }
        return result
    

    except Exception as e:
        result['keywords'] = {
            'error': str(e)
        }
        return result


In [2]:
spacing_spelling_res = {'maf_idx': 25, 'voice_to_text': ' 수고 많으십니다. 오천 힐스테이트 110동, 1005호에 사는 홍길동입니다.  퇴근하고 집에 와서 문을 열었는데 화학약품 냄새가 너무 많이 납니다. 조치 좀 취해주세요.', 'spacing_and_spelling_res': '수고 많으십니다. 오천 힐스테이트 110동 1005호에 사는 홍길동입니다. 퇴근하고 집에 와서 문을 열었는데 화학 약품 냄새가 너무 많이 나요. 조치를 좀 취해주세요.'}

In [10]:
spacing_spelling_res['spacing_and_spelling_res']

'수고 많으십니다. 오천 힐스테이트 110동 1005호에 사는 홍길동입니다. 퇴근하고 집에 와서 문을 열었는데 화학 약품 냄새가 너무 많이 나요. 조치를 좀 취해주세요.'

In [7]:
text_res = {'maf_idx': 1,
    'voice_to_text': " 수고 많으십니다. 오천 힐스테이트 110동, 1005호에 사는 홍길동입니다.  퇴근하고 집에 와서 문을 열었는데 화학약품 냄새가 너무 많이 납니다. 조치 좀 취해주세요.",
    'spacing_and_spelling_res': '수고 많으십니다. 오천 힐스테이트 110동 1005호에 사는 홍길동입니다. 퇴근하고 집에 와서 문을 열었는데 화학 약품 냄새가 너무 많이 나옵니다. 조치를 좀 취해주세요.'}

In [8]:
key_word_res = extract_information(text_res['maf_idx'], text_res)
key_word_res

{'maf_idx': 1,
 'voice_to_text': ' 수고 많으십니다. 오천 힐스테이트 110동, 1005호에 사는 홍길동입니다.  퇴근하고 집에 와서 문을 열었는데 화학약품 냄새가 너무 많이 납니다. 조치 좀 취해주세요.',
 'keywords': {'time': ['퇴근'],
  'location': None,
  'smell_type': [],
  'smell_intensity': ['너무 많이']}}